## 0.导入数据集预处理、特征工程和模型训练所需的库

In [66]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas,  numpy, textblob, string #xgboost,
#from keras.preprocessing import text, sequence
#from keras import layers, models, optimizers

## 1.1 连接数据库

In [1]:
import sqlite3
conn = sqlite3.connect("../data/database/texts.db")
print ('Opened database successfully')

Opened database successfully


## 1.2将词、诗、文言文、期刊和新闻的数据加入data列表中

In [68]:
c = conn.cursor()
data=[]
cursor = c.execute("SELECT * from ci order by id asc limit 4000")
#cursor = c.execute("SELECT * from ci")
for row in cursor:
    data.append(row)
    
cursor=c.execute("SELECT * FROM poet order by id asc limit 4000")
#cursor=c.execute("SELECT * FROM poet")
for row in cursor:
    data.append(row)
    
cursor=c.execute("SELECT * FROM classical order by id asc limit 4000")
#cursor=c.execute("SELECT * FROM classical")
for row in cursor:
    data.append(row)
    
cursor=c.execute("SELECT * FROM journal order by id asc limit 4000")
#cursor=c.execute("SELECT * FROM journal")
for row in cursor:
    data.append(row)
    
cursor=c.execute("SELECT * FROM news order by id asc limit 4000")
#cursor=c.execute("SELECT * FROM news")
for row in cursor:
    data.append(row)

In [69]:
print(data[10000:10002])

[(10002002, '    大夫士见于国君，君若劳之，则还辟，再拜稽首。君若迎拜，则还辟不敢答 拜。大夫士相见，虽贵贱不敌，主人敬客，则先拜客；客敬主人，则先拜主人。 凡非吊丧，非见国君，无不答拜者。大夫见于国君，国君拜其辱。士见于大夫， 大夫拜其辱。同国始相见，主人拜其辱。君于士，不答拜也，非其臣，则答拜之。 大夫于其臣，虽贱，必答拜之。男女相答拜也。\n', 'classical', 'c_list_10.txt'), (10002003, '    国君春田不围泽，大夫不掩群，士不取は卵。岁凶，年谷不登，君膳不祭肺， 马不食谷，驰道不除，祭事不县。大夫不食粱，士饮酒不乐。君无故，玉不去身； 大夫无故不彻县，士无故不彻琴瑟。\n', 'classical', 'c_list_10.txt')]


## 1.3 建立拆分训练数据集

In [70]:
train_data=[]
train_target=[]
for i in range(0,len(data)):
    train_data.append(data[i][1])
    train_target.append(data[i][2])
print(train_data[:3])
print(train_target[15000:15003])

['气和玉烛，睿化著鸿明。 缇管一阳生。 郊盛礼燔柴毕，旋轸凤凰城。 森罗仪卫振华缨。 载路溢欢声。 皇图大业超前古，垂象泰阶平。 岁时丰衍，九土乐升平。 睹寰海澄清。 道高尧舜垂衣治，日月并文明。 嘉禾甘露登歌荐，云物焕祥经。 兢兢惕惕持谦德，未许禅云亭。', '严夜警，铜莲漏迟迟。 清禁肃，森陛戟，羽卫俨皇闱。 角声励，钲鼓攸宜。 金管成雅奏，逐吹逶迤。 荐苍璧，郊祀神祗。 属景运纯禧。 京坻丰衍，群材乐育，诸侯述职，盛德服蛮夷。 殊祥萃，九苞丹凤来仪。 膏露降，和气洽，三秀焕灵芝。 鸿猷播，史册相辉。 张四维。 卜世永固丕基。 敷玄化，荡荡无为。 合尧舜文思。 混并寰宇，休牛归马，销金偃革，蹈咏庆昌期。', '承宝运，驯致隆平。 鸿庆被寰瀛。 时清俗阜，治定功成。 遐迩咏由庚。 严郊祀，文物声明。 会天正、星拱奏严更。 布羽仪簪缨。 宸心虔洁，明德播惟馨。 动苍冥。 神降享精诚。 燔柴半，万乘移天仗，肃銮辂旋衡。 千官云拥，群后葵倾。 玉帛旅明庭。 韶荐，金奏谐声。 集休亨。 皇泽浃黎庶，普率洽恩荣。 仰钦元后，睿圣贯三灵。 万邦宁。 景贶福千龄。']
['modern', 'modern', 'modern']


#### 测试集和训练集的切分比例为0.3

In [71]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data, train_target,test_size=0.3,random_state=0) 

# 2.特征工程 + 模型训练

## 2.0.1 加入jieba分词

In [72]:
import jieba
import numpy as np
import pandas as pd

def fenci(train_data):
    words_df = ' '.join(jieba.cut(train_data))
    return words_df

x_train_fenci = []
for i in range(len(x_train)):
    x_train_fenci.append(fenci(x_train[i]))
x_test_fenci = []
for i in range(len(x_test)):
    x_test_fenci.append(fenci(x_test[i]))
x_test_fenci[:1]

['中国 雅虎 侠客 平台 仅 提供 信息 存储空间 服务 ， 其 内容 均 由 上网 用户 提供 。 中国 雅虎 提示 您 尊重 权利 人 之 权利 。 根据 相关 法律 规定 ， 部分 内容 可能 应 权利 人 通知 未予 显示 ， 请点 这里 查看 。 \n']

## 2.0.2 引入停用词

In [2]:
infile = open("../data/stopwords_cn.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

## 2.1 计数向量作为特征

计数向量是数据集的矩阵表示，其中每行代表来自语料库的文档，每列表示来自语料库的术语，并且每个单元格表示特定文档中特定术语的频率计数：

In [74]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
vector = cv.fit_transform(x_train_fenci)
vector.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

#### 词汇表示

In [75]:
cv.vocabulary_

{'未得路': 75391,
 '莫离': 112526,
 '本位': 75608,
 '得势': 56906,
 '争先': 9838,
 '入白': 19772,
 '登道': 95186,
 '烽交': 89514,
 '紫塞途': 103227,
 '金入': 127693,
 '客军': 46656,
 '大败': 41390,
 '野有': 127578,
 '本文': 75677,
 '使用': 16333,
 '中国': 6473,
 '出口': 24124,
 '国外': 37360,
 '增加值': 39306,
 'fvar': 619,
 '测度': 85234,
 'gvc': 639,
 '嵌入': 51002,
 '程度': 100300,
 '考察': 107186,
 '核心': 78258,
 '影响': 56340,
 '因素': 37029,
 '产品': 11467,
 '遭遇': 125731,
 '反倾销': 30692,
 '状况': 90914,
 '遭受': 125722,
 '贸易': 119864,
 '伙伴': 14545,
 '诉讼': 117567,
 '如果': 43516,
 '当局': 55964,
 '确认': 98404,
 '倾销': 18021,
 '损害': 65118,
 '存在': 44853,
 '征收': 56576,
 '反倾销税': 30693,
 '施以': 69655,
 '其他': 21265,
 '限制': 131074,
 '即使': 29631,
 '经过': 104706,
 '调查': 118575,
 '最终': 74209,
 '实施': 46494,
 '措施': 65942,
 '发起': 31039,
 '申诉': 94019,
 '进行': 123770,
 '过程': 123059,
 '一定': 1310,
 '受到': 31211,
 '负面影响': 119325,
 '因此': 37006,
 '无论是': 70518,
 '还是': 123480,
 '相关': 96466,
 '指控': 64682,
 '减少': 23348,
 '根据': 78304,
 '含义': 34080,
 '当某': 56010,
 '行业': 11466

In [120]:
vectorizer = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}',stop_words=stopwords, max_features=5000)
vectorizer.fit(x_train_fenci)

xtrain_count = vectorizer.transform(x_train_fenci)
xtest_count = vectorizer.transform(x_test_fenci)

### 使用朴素贝叶斯模型训练，并得到训练分数

In [111]:
#from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
#模型训练
classifier.fit(vectorizer.transform(x_train_fenci), y_train)
#使用训练好的模型进行预测
classifier.score(vectorizer.transform(x_test_fenci), y_test)

0.8618333333333333

## 2.2 TF-IDF向量作为特征

TF-IDF的分数代表了词语在文档和整个语料库中的相对重要性。TF-IDF分数由两部分组成：第一部分是计算标准的词语频率（TF），第二部分是逆文档频率（IDF）。其中计算语料库中文档总数除以含有该词语的文档数量，然后再取对数就是逆文档频率。



TF(t)=（该词语在文档出现的次数）/（文档中词语的总数）

IDF(t)= log_e（文档总数/出现该词语的文档总数）

TF-IDF向量可以由不同级别的分词产生（单个词语，词性，多个词（n-grams））


#### （1）词语级tf-idf：矩阵代表了每个词语在不同文档中的TF-IDF分数

In [98]:
#词语级tf-idf,使用tf-idf把文本转为向量
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}',stop_words=stopwords, max_features=7000, lowercase = False)
tfidf_vect.fit(x_train_fenci)
xtrain_tfidf = tfidf_vect.transform(x_train_fenci)
xtest_tfidf = tfidf_vect.transform(x_test_fenci)

#模型训练
classifier.fit(xtrain_tfidf, y_train)
#利用训练好的模型测试
classifier.score(xtest_tfidf, y_test)

0.8648333333333333

#### （2）ngram 级tf-idf：N-grams是多个词语在一起的组合，这个矩阵代表了N-grams的TF-IDF分数。

In [108]:
# ngram 级tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}',stop_words=stopwords, ngram_range=(1,2), max_features=7000, lowercase = False)
tfidf_vect_ngram.fit(x_train_fenci)
xtrain_tfidf_ngram = tfidf_vect_ngram.transform(x_train_fenci)
xtest_tfidf_ngram = tfidf_vect_ngram.transform(x_test_fenci)

#模型训练
classifier.fit(xtrain_tfidf_ngram, y_train)
#利用训练好的模型测试
classifier.score(xtest_tfidf_ngram, y_test)

0.8623333333333333

#### （3）词性级tf-idf：矩阵代表了语料中多个词性的TF-IDF分数

In [105]:
#词性级tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(x_train_fenci)
xtrain_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(x_train_fenci)
xtest_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(x_test_fenci)

#模型训练
classifier.fit(xtrain_tfidf_ngram_chars, y_train)
#利用训练好的模型测试
classifier.score(xtest_tfidf_ngram_chars, y_test)

C:\Users\jojo\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:501: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


0.9326666666666666

# 3 构建模型

#### 之前一直都是用的朴素贝叶斯模型，那下面就换一个模型玩玩哈（没错就是线性回归模型）

In [118]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    if is_neural_net:
        predictions = predictions.argmax(axis=-1)

    return metrics.accuracy_score(predictions, y_test)

In [122]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, y_train, xtest_count)
print("LR, Count Vectors: ", accuracy)

#特征为词语级别TF-IDF向量的线性分类器
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, y_train, xtest_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

#特征为多个词语级别TF-IDF向量的线性分类器
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram)
print("LR, N-Gram Vectors: ", accuracy)

#特征为词性级别TF-IDF向量的线性分类器
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, y_train, xtest_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

C:\Users\jojo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR, Count Vectors:  0.8948333333333334


C:\Users\jojo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR, WordLevel TF-IDF:  0.9093333333333333


C:\Users\jojo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR, N-Gram Vectors:  0.907
LR, CharLevel Vectors:  0.9566666666666667


#### 把朴素贝叶斯模型训练的结果一览呈现一下

In [123]:
#特征为计数向量的朴素贝叶斯
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, y_train, xtest_count)
print("NB, Count Vectors: ", accuracy)

#特征为词语级别TF-IDF向量的朴素贝叶斯
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, y_train, xtest_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

#特征为多个词语级别TF-IDF向量的朴素贝叶斯
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy )

#特征为词性级别TF-IDF向量的朴素贝叶斯
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, y_train, xtest_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.8618333333333333
NB, WordLevel TF-IDF:  0.8648333333333333
NB, N-Gram Vectors:  0.8623333333333333
NB, CharLevel Vectors:  0.9326666666666666


# 目前仍存在的问题

## 1、没有跑遍所有的数据，好像加到5000（一共250000）就会崩掉
## 2、没有加入验证集
## 3、必须用jieba分词，否则准确率没了，但是cqk说最好不jieba？
## 4、还要引入老师提的内个要求，就是输入一段文本返回类别这个功能